## add Meta to all resources

- fetch json files from source directory using pathlib
- add meta element
- validate
- write back to directory pathlib

### Imports

In [131]:
from json import load, dumps, loads
from requests import get, post, put
from IPython.display import display, Markdown, HTML
from pprint import pprint
from pathlib import Path, WindowsPath

### globals  (target_dir)

In [132]:
mac = '/Users'
pc = '//ERICS-AIR-2'
base = 'ehaas/Documents/FHIR/Davinci-DEQM'
examples = 'input/examples'
#out_path ='r4'  #local output dir
out_path = f'{pc}/{base}/{examples}'  #DEQM source

out_path

'//ERICS-AIR-2/ehaas/Documents/FHIR/Davinci-DEQM/input/examples'

### list all file in dir

In [133]:
"\\ERICS-AIR-2\ehaas\Documents\FHIR\Davinci-DEQM\input\examples"

p= Path(out_path)
print(p)
print(p.is_dir())
files = [x for x in p.glob('*.json') if x.is_file()]

files

\\ERICS-AIR-2\ehaas\Documents\FHIR\Davinci-DEQM\input\examples
True


[WindowsPath('//ERICS-AIR-2/ehaas/Documents/FHIR/Davinci-DEQM/input/examples/Bundle-multiple-indv-mrp-obs-report.json'),
 WindowsPath('//ERICS-AIR-2/ehaas/Documents/FHIR/Davinci-DEQM/input/examples/Bundle-multiple-indv-mrp-task-report.json'),
 WindowsPath('//ERICS-AIR-2/ehaas/Documents/FHIR/Davinci-DEQM/input/examples/Bundle-multiple-summ-report.json'),
 WindowsPath('//ERICS-AIR-2/ehaas/Documents/FHIR/Davinci-DEQM/input/examples/Bundle-single-indv-col-measurereport.json'),
 WindowsPath('//ERICS-AIR-2/ehaas/Documents/FHIR/Davinci-DEQM/input/examples/Bundle-single-indv-mrp-obs-report.json'),
 WindowsPath('//ERICS-AIR-2/ehaas/Documents/FHIR/Davinci-DEQM/input/examples/Bundle-single-indv-mrp-task-report.json'),
 WindowsPath('//ERICS-AIR-2/ehaas/Documents/FHIR/Davinci-DEQM/input/examples/Bundle-single-indv-vte-measurereport.json'),
 WindowsPath('//ERICS-AIR-2/ehaas/Documents/FHIR/Davinci-DEQM/input/examples/Coverage-coverage01.json'),
 WindowsPath('//ERICS-AIR-2/ehaas/Documents/FHIR/Davinci

### Write to file

In [134]:
 def write_out(name, r):
    #out = Path() / 'out' / name
    out = p / name
    print( f'writing file to {out}')
    out.write_text(dumps(r, indent = 4))   

### Validate File

In [135]:
# *********************** validate Resource ********************************

def validate(r_dict):

    #fhir_test_server = 'http://test.fhir.org/r4'
    fhir_test_server = 'http://hapi.fhir.org/baseR4'
    #fhir_test_server ='http://wildfhir4.aegis.net/fhir4-0-1'
    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }

    r = post(f'{fhir_test_server}/{r_dict["resourceType"]}/$validate', headers = headers, data = dumps(r_dict))
    display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))

### Add meta source

- iterate
- convert to dict
- update meta if present else add meta and update
  - Bundle iterate through 
  - Parameterss iterate through

In [136]:
def add_meta(r):
    r['meta']={}
    return r


def update_meta(meta):
    meta['source'] = 'http://example.org/fhir/server'
    return meta



for x in p.glob('*.json'):
    if x.is_file():
        r_dict = loads(x.read_text())
        print(f"file = {x}\n Type = {r_dict['resourceType']}")
        try:
            m = r_dict['meta']  
            print(f"meta = {m}")
        except KeyError:  # add meta
            print(f"{r_dict['resourceType']} missing meta\n")
            r_dict = add_meta(r_dict)
            m = r_dict['meta']  
        m = update_meta(m)
        if r_dict['resourceType'] == "Bundle":
            for y in r_dict['entry']:
                r_entry = y['resource']
                try:
                      m = r_entry['meta']  
                      print(f"Entry = {r_entry ['resourceType']}\n meta = {r_entry['meta']}")
                except KeyError:  # add meta
                      print(f"Entry = {r_entry ['resourceType']} missing meta\n") 
                      r_entry = add_meta(r_entry)
                      m = r_entry['meta']                   
                m = update_meta(m)
        if r_dict['resourceType'] == "Parameters":
            for y in r_dict['parameter']:
                r_param = y['resource']
                try:
                      m = r_param['meta']
                      print(f"Entry = {r_param ['resourceType']}\n meta = {r_entry['meta']}")
                except KeyError:  # add meta
                      print(f"Entry = {r_param ['resourceType']} missing meta\n")  
                      r_param = add_meta(r_param)
                      m = r_param['meta']
                m = update_meta(m)
                            
    validate(r_dict)  
    write_out(x.name, r_dict)      

file = \\ERICS-AIR-2\ehaas\Documents\FHIR\Davinci-DEQM\input\examples\Bundle-multiple-indv-mrp-obs-report.json
 Type = Bundle
Bundle missing meta

Entry = Practitioner
 meta = {'profile': ['http://hl7.org/fhir/us/davinci-deqm/StructureDefinition/practitioner-deqm']}
Entry = Patient
 meta = {'lastUpdated': '2019-03-06T19:03:03.000+00:00', 'profile': ['http://hl7.org/fhir/us/qicore/StructureDefinition/qicore-patient'], 'versionId': '4'}
Entry = Coverage
 meta = {'lastUpdated': '2019-02-11T14:19:25.000+00:00', 'profile': ['http://hl7.org/fhir/us/davinci-deqm/StructureDefinition/coverage-deqm'], 'versionId': '2'}
Entry = MeasureReport
 meta = {'profile': ['http://hl7.org/fhir/us/davinci-deqm/StructureDefinition/indv-measurereport-deqm']}
Entry = MeasureReport
 meta = {'profile': ['http://hl7.org/fhir/us/davinci-deqm/StructureDefinition/indv-measurereport-deqm']}
Entry = Organization
 meta = {'lastUpdated': '2019-03-06T19:03:03.000+00:00', 'profile': ['http://hl7.org/fhir/us/davinci-deqm/St

ERROR,[],Failed to call access method: org.springframework.dao.InvalidDataAccessApiUsageException: Can't fetch resource type: Measure; nested exception is java.lang.IllegalArgumentException: Can't fetch resource type: Measure


writing file to \\ERICS-AIR-2\ehaas\Documents\FHIR\Davinci-DEQM\input\examples\Bundle-multiple-indv-mrp-obs-report.json
file = \\ERICS-AIR-2\ehaas\Documents\FHIR\Davinci-DEQM\input\examples\Bundle-multiple-indv-mrp-task-report.json
 Type = Bundle
Bundle missing meta

Entry = MeasureReport
 meta = {'profile': ['http://hl7.org/fhir/us/davinci-deqm/StructureDefinition/indv-measurereport-deqm']}
Entry = Encounter
 meta = {'profile': ['http://hl7.org/fhir/us/qicore/StructureDefinition/qicore-encounter']}
Entry = Organization
 meta = {'lastUpdated': '2019-03-06T19:03:03.000+00:00', 'profile': ['http://hl7.org/fhir/us/davinci-deqm/StructureDefinition/organization-deqm'], 'versionId': '3'}
Entry = MeasureReport
 meta = {'profile': ['http://hl7.org/fhir/us/davinci-deqm/StructureDefinition/indv-measurereport-deqm']}
Entry = Practitioner
 meta = {'profile': ['http://hl7.org/fhir/us/davinci-deqm/StructureDefinition/practitioner-deqm']}
Entry = Patient
 meta = {'lastUpdated': '2019-02-07T18:47:34.0

ERROR,[],Failed to call access method: org.springframework.dao.InvalidDataAccessApiUsageException: Can't fetch resource type: Measure; nested exception is java.lang.IllegalArgumentException: Can't fetch resource type: Measure


writing file to \\ERICS-AIR-2\ehaas\Documents\FHIR\Davinci-DEQM\input\examples\Bundle-multiple-indv-mrp-task-report.json
file = \\ERICS-AIR-2\ehaas\Documents\FHIR\Davinci-DEQM\input\examples\Bundle-multiple-summ-report.json
 Type = Bundle
Bundle missing meta

Entry = MeasureReport missing meta

Entry = Organization
 meta = {'lastUpdated': '2019-02-21T01:47:54.000+00:00', 'profile': ['http://hl7.org/fhir/us/davinci-deqm/StructureDefinition/organization-deqm'], 'versionId': '4'}
Entry = MeasureReport
 meta = {'profile': ['http://hl7.org/fhir/us/davinci-deqm/StructureDefinition/summary-measurereport-deqm']}
Entry = MeasureReport
 meta = {'profile': ['http://hl7.org/fhir/us/davinci-deqm/StructureDefinition/summary-measurereport-deqm']}


...

writing file to \\ERICS-AIR-2\ehaas\Documents\FHIR\Davinci-DEQM\input\examples\Task-task01.json
file = \\ERICS-AIR-2\ehaas\Documents\FHIR\Davinci-DEQM\input\examples\Task-task02.json
 Type = Task
meta = {'profile': ['http://hl7.org/fhir/us/qicore/StructureDefinition/qicore-task']}


WARNING,"[Task, Line 1, Col 2]",dom-6: A resource should have narrative for robust management [text.div.exists()]
ERROR,"[Task.meta.profile[0], Line 1, Col 2]","Profile reference ""http://hl7.org/fhir/us/qicore/StructureDefinition/qicore-task"" could not be resolved, so has not been checked"


writing file to \\ERICS-AIR-2\ehaas\Documents\FHIR\Davinci-DEQM\input\examples\Task-task02.json
file = \\ERICS-AIR-2\ehaas\Documents\FHIR\Davinci-DEQM\input\examples\Task-task03.json
 Type = Task
meta = {'profile': ['http://hl7.org/fhir/us/qicore/StructureDefinition/qicore-task']}


WARNING,"[Task, Line 1, Col 2]",dom-6: A resource should have narrative for robust management [text.div.exists()]
ERROR,"[Task.meta.profile[0], Line 1, Col 2]","Profile reference ""http://hl7.org/fhir/us/qicore/StructureDefinition/qicore-task"" could not be resolved, so has not been checked"


writing file to \\ERICS-AIR-2\ehaas\Documents\FHIR\Davinci-DEQM\input\examples\Task-task03.json
